In [4]:
import numpy as np
import os
import pandas as pd
import shutil
import nipype.interfaces.fsl as fsl
import matplotlib.pyplot as plt
from pathlib import Path
import nibabel as nib



Apply BET mask to the T1 brain registered to FLAIR space for consistency.

In [ ]:
apply_mask_BET = fsl.ApplyMask()
apply_mask_BET.inputs.mask_file = '/home/ela/Documents/B-RAPIDD/B-RAP_0100/3D-FLAIR/brain_nifti/masks/B-RAP_0100_05_D1.nii'
apply_mask_BET.inputs.out_file = '/home/ela/Documents/B-RAPIDD/B-RAP_0100/3D-FLAIR/registered_T1/B-RAP_0100_01_E1_BET.nii.gz'
T1_bet_res = apply_mask_BET.run()
print('Extracted T1 brain')


After applying FAST, load in the mask for CSF and apply it to the chosen variance map.
(B-RAPP_0100 Affine variance)

In [28]:
csf_mask = nib.load('/home/ela/Documents/B-RAPIDD/B-RAP_0100/3D-FLAIR/registered_T1/B-RAP_0100_01_E1_CSF_mask.nii.gz').get_fdata()
variance_map = nib.load('/home/ela/Documents/B-RAPIDD/B-RAP_0100/3D-FLAIR/variance_maps/affine/intrasubnormalised_all_timepoints.nii.gz').get_fdata()
out_file = '/home/ela/Documents/B-RAPIDD/B-RAP_0100/3D-FLAIR/variance_maps/affine/no_csf/intrasubnormalised_all_timepoints_no_csf.nii.gz'


# invert the mask to keep non-csf
inv_csf_mask = np.logical_not(csf_mask)*1 # * 1 converts to 1 or 0 from a true/false to allow multiplication

variance_map_no_csf = np.multiply(variance_map,inv_csf_mask) # element-wise multiplication

print(variance_map_no_csf.shape)

# save the map
var_no_csf_nifti =  nib.Nifti1Image(variance_map_no_csf, affine=np.eye(4))
nib.save(var_no_csf_nifti, out_file)

(176, 232, 256)
